In [ ]:
!pip install python-dotenv
!pip3 install requests

In [ ]:
import numpy as np
import pandas as pd
import requests as r
import time
import os
from dotenv import load_dotenv
%load_ext dotenv
%dotenv

In [ ]:
test = pd.read_csv('./datas/test.csv', index_col=0)
train = pd.read_csv('./datas/train.csv',  index_col=0)
jeju_life = pd.read_csv('./datas/jeju_financial_life_data.csv')

test.head()


In [ ]:
# train과 test를 합칩니다.
data = pd.concat([train, test])

data.head()



In [ ]:
# test, train, jeju_life에 각각 location, dong, si 칼럼을 추가한다.

data['location'] = ''
data['dong'] = ''
data['si'] = ''

jeju_life['location'] = ''
jeju_life['dong'] = ''
jeju_life['si'] = ''


In [ ]:
# 각각 location을 만드는데, 이는 위도와 경도를 ,로 합친 것이다.
# jeju_life['location'] = jeju_life['x_axis'].astype(str).str[:10] + ',' + jeju_life['y_axis'].astype(str).str[:10]

data['location'] = data['longitude'].astype(str).str[:10]  + ',' + data['latitude'].astype(str).str[:10] 

In [ ]:
data['latitude']

In [ ]:
jeju_life['location'] = jeju_life['x_axis'].astype(str).str[:10] + ',' + jeju_life['y_axis'].astype(str).str[:10]
jeju_life.head()

url = "curl "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={입력_좌표}&sourcecrs={좌표계}&orders={변환_작업_이름}&output={출력_형식}" \
	-H "X-NCP-APIGW-API-KEY-ID: {애플리케이션 등록 시 발급받은 client id값}" \
	-H "X-NCP-APIGW-API-KEY: {애플리케이션 등록 시 발급받은 client secret값}" -v"

In [ ]:
%dotenv ./.env

jeju_finantial_life_data (외부데이터) 사용하여 피쳐 생성

train, test 데이터의 'latitude', 'longitude'열, jeju_financial_life_data의 'X_AXIS', 'Y_AXIS'열을 지오코딩 프로그램을 사용하여 주소로 변환하였다. 변환한 주소를 전처리 하여 동 기준으로 두 데이터를 합쳐주었다. 그 뒤 파생변수를 생성하였다. data_address.csv는 data의 주소가, life_address.csv는 jeju_finantial_life_data의 주소가 담겨있다.




In [ ]:
#coords = "127.1054321,37.3595963"

def reverse_geocode(coords):
	url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords=%s&orders=legalcode&output=json" %coords

	response = r.get(url, 
					headers={"X-NCP-APIGW-API-KEY-ID": os.getenv("CLIENT_ID"), 
			  				"X-NCP-APIGW-API-KEY": os.getenv("CLIENT_SECRET")})
	data = response.json()
	if (data['status']['code'] == 100):
		print("error", coords)

	dong = data['results'][0]['region']['area3']['name']
	si = data['results'][0]['region']['area2']['name'].split()[0]
	
	return {"si":si, "dong":dong}


In [ ]:
data.head()

In [ ]:
# 자 이제, data에 있는 location을 이용해서, 동과 시를 구해보자.
# 64만개의 데이터를 한 번에 처리하는 것은 무리이다. 따라서 batch로 50000개씩 나누어서 처리하도록 하자.
# 그리고 50000개씩 나누어 처리할때, 10000개씩 멀티프로세싱을 이용해서 처리하도록 하자.

# 처리완료된 데이터는 파일로 따로 처리한다.

import multiprocess

batch = 50000

def thread_function(name, start, end):
	end = min(end, len(data))
	for i in range(start, end):
		print(i)
		result = reverse_geocode(data['location'][i])
		data['si'][i] = result['si']
		data['dong'][i] = result['dong']





In [ ]:
import threading

jobs = []

def batch_threading(step, batchSize):
	batch_start = (step -1)* (batchSize)
	batch_end = step * batchSize
	

	# batch 크기는 정해졌으므로, 여기서 4개의 멀티프로세싱을 이용해서 처리하도록 하자.
	for i in range(4):
		start = batch_start + int(50000 / 4 * i)
		end = batch_end + int(50000 / 4 * (i+1))
		p = threading.Thread(target=thread_function, args=(i, start, end))
		jobs.append(p)
		p.start()
		time.sleep(1)

	for proc in jobs:
		proc.join()

	# 처리된 데이터만 잘라내서 서장

	data.iloc[batch_start:batch_end, :].to_csv("./datas/data%d.csv" %step)
	
	

In [ ]:
batch_threading(12, batch)